In [1]:
import numpy as np

from pyboolnet.external.bnet2primes import bnet_text2primes
from pyboolnet.state_transition_graphs import primes2stg

from transition_matrix import get_transition_matrix
from attractors import get_predicted_attractors
from matrix_operations import nsquare, compress_matrix, expand_matrix, get_rms_diff, get_dkl, get_confusion_matrix
from grouping import sd_grouping
from basins import get_convergence_matrix

np.set_printoptions(linewidth=1000, precision=3, suppress=True)

In [2]:
bnet = """
A, A | B & C
B, B & !C
C, B & !C | !C & !D | !B & C & D
D, !A & !B & !C & !D | !A & C & D
"""

update = "asynchronous"

DEBUG = True

In [3]:
primes = bnet_text2primes(bnet)
primes = {key: primes[key] for key in sorted(primes)}
stg = primes2stg(primes, update)

In [4]:
T = get_transition_matrix(stg, update=update, DEBUG=DEBUG)
T_inf = nsquare(T, 20, DEBUG=DEBUG)

In [5]:
sd_indices = sd_grouping(bnet, DEBUG=DEBUG)
Tsd = compress_matrix(T, sd_indices, DEBUG=DEBUG)
Tsd_inf = nsquare(Tsd, 20, DEBUG=DEBUG)
Tsd_inf_expanded = expand_matrix(Tsd_inf, sd_indices, DEBUG=DEBUG)

In [6]:
predicted_attractor_indices = get_predicted_attractors(Tsd, sd_indices, as_indices=True, DEBUG=DEBUG)

T_convergence = get_convergence_matrix(T_inf, predicted_attractor_indices, DEBUG=DEBUG)
Tsd_convergence = get_convergence_matrix(Tsd_inf_expanded, predicted_attractor_indices, DEBUG=DEBUG)

print("T_convergence")
print(T_convergence)

print("Tsd_convergence")
print(Tsd_convergence)

T_convergence
[[1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.  0.5]
 [0.3 0.2 0.5]
 [0.5 0.  0.5]
 [0.1 0.4 0.5]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]
 [0.  0.  1. ]]
Tsd_convergence
[[1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.   1.   0.  ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.25 0.25 0.5 ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]]


In [7]:
rmsd = get_rms_diff(T_convergence, Tsd_convergence, partial=True, DEBUG=DEBUG)
dkl = get_dkl(T_convergence, Tsd_convergence, partial=True, DEBUG=DEBUG)

print("rmsd")
print(rmsd)

print("dkl")
print(dkl)

rmsd
0.0790569415042095
dkl
0.7995873158461685


In [8]:
TP, FP, TN, FN = get_confusion_matrix(T_convergence, Tsd_convergence, partial=True, DEBUG=DEBUG)

print("TP, FP, TN, FN")
print(TP, FP, TN, FN)

TP, FP, TN, FN
22 2 24 0
